In [ ]:
import numpy
from matplotlib import pyplot
%matplotlib inline

from matplotlib import rcParams
rcParams['font.family'] = 'serif'
rcParams['font.size'] = 14

In [ ]:
def eulerstep(state, rhs, dt):
    '''Update a state to the next time increment using Euler's method.
    
    Arguments
    ---------
    state : array of dependent variables
    rhs   : function that computes the RHS of the DiffEq
    dt    : float, time increment
    
    Returns
    -------
    next_state : array, updated after one time increment'''
    
    next_state = state + rhs(state) * dt
    return next_state

### Spring-mass system

A prototypical mechanical system is a mass $m$ attached to a spring, in the simplest case without friction. The elastic constant of the spring, $k$, determines the restoring force it will apply to the mass when displaced by a distance $x$. The system then oscillates back and forth around its position of equilibrium.

<img src="images/spring-mass.png" style="width: 400px;"/> 
#### Simple spring-mass system, without friction.

Newton's law applied to the friction-less spring-mass system is:

\begin{equation}
-k x = m \ddot{x}
\end{equation}

Introducing the parameter $\omega = \sqrt{k/m}$, the equation of motion is rewriten as:

\begin{equation}
\ddot{x} + \omega^2 x = 0
\end{equation}

where a dot above a dependent variable denotes the time derivative. This is a second-order differential equation for the position $x$, having a known analytical solution that represents _simple harmonic motion_:

$x(t) = x_0 \cos(\omega t)$

The solution represents oscillations with period $P = 2 \pi/ \omega $  (the time between two peaks), and amplitude $x_0$.

### System in vector form

It's useful to write a second-order differential equation as a set of two first-order equations: in this case, for position and velocity, respectively:

\begin{eqnarray}
\dot{x} &=& v \nonumber\\
\dot{v} &=& -\omega^2 x
\end{eqnarray}

Like we did in [Lesson 2](http://go.gwu.edu/engcomp3lesson2) of this module, we write the state of the system as a two-dimensional vector,

\begin{equation}
\mathbf{x} = \begin{bmatrix}
x \\ v
\end{bmatrix},
\end{equation}

and the differential equation in vector form:

\begin{equation}
\dot{\mathbf{x}} = \begin{bmatrix}
v \\ -\omega^2 x
\end{bmatrix}.
\end{equation}

Several advantages come from writing the differential equation in vector form, both  theoretical and practical. In the study of dynamical systems, for example, the state vector lives in a state space called the _phase plane_, and many things can be learned from studying solutions to differential equations graphically on a phase plane.

Practically, writing the equation in vector form results in more general, compact code. Let's write a function to obtain the right-hand side of the spring-mass differential equation, in vector form.

In [ ]:
def springmass(state):
    '''Computes the right-hand side of the spring-mass differential 
    equation, without friction.
    
    Arguments
    ---------   
    state : array of two dependent variables [x v]^T
    
    Returns 
    -------
    derivs: array of two derivatives [v - ω*ω*x]^T
    '''
    
    derivs = numpy.array([state[1], -ω**2*state[0]])
    return derivs

In [ ]:
ω = 2
period = 2*numpy.pi/ω
dt = period/20  # we choose 20 time intervals per period 
T = 3*period    # solve for 3 periods
N = round(T/dt)


In [ ]:
t = numpy.linspace(0, T, N)

In [ ]:
print(N)
print(dt)

In [ ]:
x0 = 2    # initial position
v0 = 0    # initial velocity

In [ ]:
#initialize solution array
num_sol = numpy.zeros([N,2])

In [ ]:
#Set intial conditions
num_sol[0,0] = x0
num_sol[0,1] = v0

In [ ]:
for i in range(N-1):
    num_sol[i+1] = eulerstep(num_sol[i], springmass, dt)

Let's compute the position with respect to time using the known analytical solution:

In [ ]:
x_an = x0*numpy.cos(ω * t)

In [ ]:
fig = pyplot.figure(figsize=(6,4))

pyplot.plot(t, num_sol[:, 0], linewidth=2, linestyle='--', label='Numerical solution')
pyplot.plot(t, x_an, linewidth=1, linestyle='-', label='Analytical solution')
pyplot.grid()
pyplot.xlabel('Time [s]')
pyplot.ylabel('$x$ [m]')
pyplot.title('Spring-mass system \n');

**Exercise: ** Reduce the discretization parameter, `dt`, and see if the results become more accurate. Try `P/40`,  `P/160` and  `P/2000`.

Although the 2000 iterval per oscilation period seem enough to have an accurate solution. Let's see what happen if we increase the time of simulation, for example to 20 periods. (Run the case again)

What do you see?

There is a little growth of the amplitude, which becomes significant over time. The conclusion is that the Forward Euler method has a fundamental problem with its growing amplitudes, and that a very small `dt`is needed to achieve results. Besides, the longer the simulation, the smaller the `dt` has to be.  It is certainly time to look for more effective numerical methods!


We can fix this with a little tweak. 

Until now we were doing:

\begin{eqnarray}
x(t_0) = x_0, \qquad x_{i+1} &=& x_i + \dot{x} \Delta t \nonumber\\
v(t_0) = v_0, \qquad v_{i+1} &=& v_i + (-{\omega}^2 x_i) \Delta t
\end{eqnarray}

What if in the equation of $v$ we use the $x_{i+1}$ we computed in the previous line?


\begin{eqnarray}
x(t_0) = x_0, \qquad x_{i+1} &=& x_i + \dot{x} \Delta t \nonumber\\
v(t_0) = v_0, \qquad v_{i+1} &=& v_i + (-{\omega}^2 x_{i+1}) \Delta t
\end{eqnarray}

In [ ]:
def euler_cromer(state, rhs, dt):
    '''Update a state to the next time increment using Euler-Cromer's method.
    
    Arguments
    ---------
    state : array of dependent variables
    rhs   : function that computes the RHS of the DiffEq
    dt    : float, time increment
    
    Returns
    -------
    next_state : array, updated after one time increment'''
    
    mid_state = state + rhs(state)*dt # Euler step
    mid_derivs = rhs(mid_state)       # updated derivatives
    
    next_state = numpy.array([mid_state[0], state[1] + mid_derivs[1]*dt])
    
    return next_state

In [ ]:
ω = 2
period = 2*numpy.pi/ω
dt = period/200  #if we want 200 intervals per period 
T = 4000*period #simulate for 3 periods
N = round(T/dt)


t = numpy.linspace(0, T, N)

print(N)
print(dt)

x0 = 2    # initial position
v0 = 0    # initial velocity

#initialize solution array
num_sol = numpy.zeros([N,2])

#Set intial conditions
num_sol[0,0] = x0
num_sol[0,1] = v0

for i in range(N-1):
    num_sol[i+1] = euler_cromer(num_sol[i], springmass, dt)

In [ ]:
x_an = x0*numpy.cos(ω * t)

In [ ]:
fig = pyplot.figure(figsize=(6,4))

pyplot.plot(t[:1000], num_sol[:1000, 0], linewidth=2, linestyle='--', label='Numerical solution')
pyplot.plot(t[:1000], x_an[:1000], linewidth=1, linestyle='-', label='Analytical solution')
pyplot.grid()
pyplot.xlabel('Time [s]')
pyplot.ylabel('$x$ [m]')
pyplot.title('Spring-mass system \n');

In [ ]:
fig = pyplot.figure(figsize=(6,4))

pyplot.plot(t[-500:], num_sol[-500:, 0], linewidth=2, linestyle='--', label='Numerical solution')
pyplot.plot(t[-500:], x_an[-500:], linewidth=1, linestyle='-', label='Analytical solution')
pyplot.grid()
pyplot.xlabel('Time [s]')
pyplot.ylabel('$x$ [m]')
pyplot.title('Spring-mass system \n');